# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import os
import sqlite3
import pickle

In [30]:
#Importing additional libraries for ML
import nltk
nltk.download(['wordnet', 'punkt', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
#loading the data from database
def load_data(db_path='workspace/data/DisasterResponse.db',tablename='disaster_message_table'):
    
    """ 
    This function is to take extract data from sqlite3 database and return X and y.

        Args:
          db_path(str): database file name path
          tablename:(str): table name in the database file.
        Return:
          X(pd.DataFrame): Text messages
          y(pd.DataFrame): Category of messages out of 36 categories
    """

    conn = sqlite3.connect('DisasterResponse.db')
    df = pd.read_sql('SELECT * FROM disaster_message_table', conn)
    X = df.message.values
    y = df.iloc[:, 4:]
    y = y.astype(str)
    return X, y




### 2. Write a tokenization function to process your text data

In [32]:
def tokenize(text):
    """
    This function is to process the received messages(text) into usable form.
    it normalizes, tokenize, removes stop-words, lemmatizes, and finally stores clean tokenized word in a list
    """
    
    #Text Normalization
    text= re.sub(r"[^a-zA-Z0_9]"," ", text)
    
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # Removing stop words
    stop = stopwords.words("english")
    tokens= [w for w in tokens if w not in stop]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
 

### 2.1 Write a function to dispay the Accuracy Score

In [33]:

def publish_scores(y_test, y_pred):
    
    """ 
    This funtion dispaly the accuracy of each label after prediction using a classifier.
    y_test: Target variable of the test data
    y_pred: Predicted target value
    """
   
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [34]:
# First Pipeline using Random forest classifier
pipeline_rf = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

# Second pipeline using Naive Bayes classifier

pipeline_nb = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(MultinomialNB()))
    ])

# Second pipeline using Logistic Regression classifier

pipeline_kn = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

### 4. Perform a train test split & Train the Model

In [35]:
# perform train test split
X, y = load_data()

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Training first pipeline for Random Forest classifier
pipeline_rf.fit(X_train, y_train)

# Training second pipeline for Naive Bayes classifier
pipeline_nb.fit(X_train, y_train)

# Training third pipeline for Logistic KNeighbors Classifier
pipeline_kn.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [36]:
#Publishing score for the first pipeline with RandomForest Classifier

y_pred= pipeline_rf.predict(X_test)

publish_scores(y_test, y_pred)

Feature 1: genre
             precision    recall  f1-score   support

     direct       1.00      1.00      1.00      2543

avg / total       1.00      1.00      1.00      2543

Feature 2: related
             precision    recall  f1-score   support

          0       0.66      0.65      0.65       865
          1       0.81      0.83      0.82      1630
          2       0.61      0.40      0.48        48

avg / total       0.75      0.76      0.76      2543

Feature 3: request
             precision    recall  f1-score   support

          0       0.85      0.89      0.87      1681
          1       0.77      0.69      0.73       862

avg / total       0.82      0.83      0.82      2543

Feature 4: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2538
          1       0.00      0.00      0.00         5

avg / total       1.00      1.00      1.00      2543

Feature 5: aid_related
             precision    recall  f1-score   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2543

avg / total       1.00      1.00      1.00      2543

Feature 12: water
             precision    recall  f1-score   support

          0       0.98      0.99      0.99      2343
          1       0.90      0.74      0.81       200

avg / total       0.97      0.97      0.97      2543

Feature 13: food
             precision    recall  f1-score   support

          0       0.96      0.98      0.97      2160
          1       0.89      0.76      0.82       383

avg / total       0.95      0.95      0.95      2543

Feature 14: shelter
             precision    recall  f1-score   support

          0       0.94      0.99      0.97      2295
          1       0.79      0.46      0.58       248

avg / total       0.93      0.94      0.93      2543

Feature 15: clothing
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      2525
          

In [37]:
#Publishing score for the second pipeline with NaiveBayes Classifier

y_pred= pipeline_nb.predict(X_test)

publish_scores(y_test, y_pred)

Feature 1: genre
             precision    recall  f1-score   support

     direct       1.00      1.00      1.00      2543

avg / total       1.00      1.00      1.00      2543

Feature 2: related
             precision    recall  f1-score   support

          0       0.83      0.36      0.50       865
          1       0.72      0.96      0.83      1630
          2       1.00      0.02      0.04        48

avg / total       0.77      0.74      0.70      2543

Feature 3: request
             precision    recall  f1-score   support

          0       0.84      0.90      0.87      1681
          1       0.77      0.66      0.71       862

avg / total       0.81      0.82      0.81      2543

Feature 4: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2538
          1       0.00      0.00      0.00         5

avg / total       1.00      1.00      1.00      2543

Feature 5: aid_related
             precision    recall  f1-score   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.99      1.00      0.99      2514
          1       0.00      0.00      0.00        29

avg / total       0.98      0.99      0.98      2543

Feature 17: missing_people
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      2522
          1       0.00      0.00      0.00        21

avg / total       0.98      0.99      0.99      2543

Feature 18: refugees
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      2502
          1       0.00      0.00      0.00        41

avg / total       0.97      0.98      0.98      2543

Feature 19: death
             precision    recall  f1-score   support

          0       0.97      1.00      0.99      2472
          1       0.00      0.00      0.00        71

avg / total       0.94      0.97      0.96      2543

Feature 20: other_aid
             precision    recall  f1-score   support

In [38]:
#Publishing score for the third pipeline with KNeighbors Classifier

y_pred= pipeline_kn.predict(X_test)

publish_scores(y_test, y_pred)

Feature 1: genre
             precision    recall  f1-score   support

     direct       1.00      1.00      1.00      2543

avg / total       1.00      1.00      1.00      2543

Feature 2: related
             precision    recall  f1-score   support

          0       0.63      0.60      0.61       865
          1       0.78      0.81      0.79      1630
          2       0.63      0.25      0.36        48

avg / total       0.72      0.73      0.72      2543

Feature 3: request
             precision    recall  f1-score   support

          0       0.73      0.94      0.83      1681
          1       0.75      0.33      0.46       862

avg / total       0.74      0.74      0.70      2543

Feature 4: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2538
          1       0.00      0.00      0.00         5

avg / total       1.00      1.00      1.00      2543

Feature 5: aid_related
             precision    recall  f1-score   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.94      1.00      0.97      2343
          1       0.83      0.21      0.34       200

avg / total       0.93      0.93      0.92      2543

Feature 13: food
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      2160
          1       0.83      0.26      0.39       383

avg / total       0.87      0.88      0.85      2543

Feature 14: shelter
             precision    recall  f1-score   support

          0       0.92      0.99      0.95      2295
          1       0.75      0.16      0.27       248

avg / total       0.90      0.91      0.89      2543

Feature 15: clothing
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      2525
          1       0.00      0.00      0.00        18

avg / total       0.99      0.99      0.99      2543

Feature 16: money
             precision    recall  f1-score   support

          

### 6. Improve your model
Use grid search to find better parameters. 

In [39]:
# getting the parameters for the First Pipeline
pipeline_rf.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0465e5a048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [40]:
# getting the parameters for the Second Pipeline
pipeline_nb.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0465e5a048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=N

In [41]:
# getting the parameters for the third Pipeline
pipeline_kn.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0465e5a048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, n_jobs=1, n_neighbors=5, p=2,
              weights='uniform'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_fea

In [42]:
# Creating Grid search parameters for the first model with RandomForest Classifier
params_rf = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 50, 100]
}

cv_rf = GridSearchCV(pipeline_rf, param_grid = params_rf)
cv_rf

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [43]:
# Creating Grid search parameters for the second model with NaiveBayes Classifier
params_nb = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__alpha': [.1, 1, 2]
}

cv_nb = GridSearchCV(pipeline_nb, param_grid = params_nb)

In [44]:
# Creating Grid search parameters for the third model with KNeighbors Classifier
params_kn = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__leaf_size': [20, 30, 50]
}

cv_kn = GridSearchCV(pipeline_kn, param_grid = params_kn)
cv_kn

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__leaf_size': [20, 30, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [45]:
#Fitting the first tuned model with RandomForest Classifier parameters & predicting the results

cv_rf.fit(X_train, y_train)
y_pred= cv_rf.predict(X_test)
publish_scores(y_test, y_pred)

Feature 1: genre
             precision    recall  f1-score   support

     direct       1.00      1.00      1.00      2543

avg / total       1.00      1.00      1.00      2543

Feature 2: related
             precision    recall  f1-score   support

          0       0.71      0.63      0.67       865
          1       0.80      0.87      0.83      1630
          2       0.90      0.19      0.31        48

avg / total       0.77      0.77      0.77      2543

Feature 3: request
             precision    recall  f1-score   support

          0       0.88      0.90      0.89      1681
          1       0.79      0.75      0.77       862

avg / total       0.85      0.85      0.85      2543

Feature 4: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2538
          1       0.00      0.00      0.00         5

avg / total       1.00      1.00      1.00      2543

Feature 5: aid_related
             precision    recall  f1-score   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.98      0.99      0.99      2343
          1       0.90      0.78      0.83       200

avg / total       0.97      0.98      0.97      2543

Feature 13: food
             precision    recall  f1-score   support

          0       0.97      0.98      0.98      2160
          1       0.89      0.83      0.86       383

avg / total       0.96      0.96      0.96      2543

Feature 14: shelter
             precision    recall  f1-score   support

          0       0.96      0.99      0.97      2295
          1       0.84      0.57      0.68       248

avg / total       0.94      0.95      0.94      2543

Feature 15: clothing
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      2525
          1       0.00      0.00      0.00        18

avg / total       0.99      0.99      0.99      2543

Feature 16: money
             precision    recall  f1-score   support

          

In [46]:
#Fitting the second tuned model with Naive Bayes Classifier parameters

cv_nb.fit(X_train, y_train)
y_pred= cv_nb.predict(X_test)
publish_scores(y_test, y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_

Feature 1: genre
             precision    recall  f1-score   support

     direct       1.00      1.00      1.00      2543

avg / total       1.00      1.00      1.00      2543

Feature 2: related
             precision    recall  f1-score   support

          0       0.75      0.58      0.66       865
          1       0.79      0.90      0.84      1630
          2       0.80      0.25      0.38        48

avg / total       0.78      0.78      0.77      2543

Feature 3: request
             precision    recall  f1-score   support

          0       0.87      0.86      0.86      1681
          1       0.73      0.75      0.74       862

avg / total       0.82      0.82      0.82      2543

Feature 4: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2538
          1       0.00      0.00      0.00         5

avg / total       1.00      1.00      1.00      2543

Feature 5: aid_related
             precision    recall  f1-score   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.99      1.00      1.00      2525
          1       0.00      0.00      0.00        18

avg / total       0.99      0.99      0.99      2543

Feature 16: money
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      2514
          1       0.00      0.00      0.00        29

avg / total       0.98      0.99      0.98      2543

Feature 17: missing_people
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      2522
          1       0.00      0.00      0.00        21

avg / total       0.98      0.99      0.99      2543

Feature 18: refugees
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      2502
          1       0.00      0.00      0.00        41

avg / total       0.97      0.98      0.98      2543

Feature 19: death
             precision    recall  f1-score   support

  

In [47]:
#Fitting the third tuned model with Logestic KNeighbors Classifier parameters

cv_kn.fit(X_train, y_train)
y_pred= cv_kn.predict(X_test)
publish_scores(y_test, y_pred)

Feature 1: genre
             precision    recall  f1-score   support

     direct       1.00      1.00      1.00      2543

avg / total       1.00      1.00      1.00      2543

Feature 2: related
             precision    recall  f1-score   support

          0       0.64      0.67      0.66       865
          1       0.81      0.81      0.81      1630
          2       0.62      0.10      0.18        48

avg / total       0.75      0.75      0.74      2543

Feature 3: request
             precision    recall  f1-score   support

          0       0.79      0.93      0.85      1681
          1       0.79      0.51      0.62       862

avg / total       0.79      0.79      0.77      2543

Feature 4: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2538
          1       0.00      0.00      0.00         5

avg / total       1.00      1.00      1.00      2543

Feature 5: aid_related
             precision    recall  f1-score   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.97      0.99      0.98      2343
          1       0.89      0.59      0.71       200

avg / total       0.96      0.96      0.96      2543

Feature 13: food
             precision    recall  f1-score   support

          0       0.93      0.99      0.96      2160
          1       0.89      0.56      0.69       383

avg / total       0.92      0.92      0.92      2543

Feature 14: shelter
             precision    recall  f1-score   support

          0       0.93      0.99      0.96      2295
          1       0.85      0.33      0.48       248

avg / total       0.92      0.93      0.91      2543

Feature 15: clothing
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      2525
          1       0.00      0.00      0.00        18

avg / total       0.99      0.99      0.99      2543

Feature 16: money
             precision    recall  f1-score   support

          

##### Out of 3 Models , Random forest Model is yielding best  precdiction accuracy(~95%) after using Gris search cross Validation. KNeighbour Model is least accurate out of three (91.6 %).

In [48]:
# Training the model with improved parmeters -Choosing Random forest classifier with Grid search best parameters
def main():
    
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # build pipeline
    pipeline_rf = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
         
        
    # train classifier
    pipeline_rf.fit(X_train, y_train)
    # predict on test data
    y_pred=pipeline_rf.predict(X_test)
    
    # Creating Grid search parameters for the first model with RandomForest Classifier
    params_rf = {
            'tfidf__use_idf': (True, False),
            'clf__estimator__n_estimators': [10, 50, 100]
    }

    cv_rf = GridSearchCV(pipeline_rf, param_grid = params_rf)
    
    #Fitting the tuned model with RandomForest Classifier parameters & predicting the results

    cv_rf.fit(X_train, y_train)
    y_pred= cv_rf.predict(X_test)

    # display results
    publish_scores(y_test, y_pred)
    
    return cv_rf
    


In [49]:
# Assigning the output of the main() function to a variable 
classifier_model = main()

Feature 1: genre
             precision    recall  f1-score   support

     direct       1.00      1.00      1.00      2543

avg / total       1.00      1.00      1.00      2543

Feature 2: related
             precision    recall  f1-score   support

          0       0.75      0.62      0.68       828
          1       0.83      0.89      0.86      1680
          2       0.38      0.49      0.42        35

avg / total       0.80      0.80      0.79      2543

Feature 3: request
             precision    recall  f1-score   support

          0       0.87      0.89      0.88      1613
          1       0.79      0.76      0.78       930

avg / total       0.84      0.84      0.84      2543

Feature 4: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2543

avg / total       1.00      1.00      1.00      2543

Feature 5: aid_related
             precision    recall  f1-score   support

          0       0.85      0.88      0.87  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.98      0.99      0.98      2352
          1       0.85      0.76      0.80       191

avg / total       0.97      0.97      0.97      2543

Feature 13: food
             precision    recall  f1-score   support

          0       0.96      0.98      0.97      2130
          1       0.91      0.81      0.86       413

avg / total       0.96      0.96      0.96      2543

Feature 14: shelter
             precision    recall  f1-score   support

          0       0.94      0.99      0.96      2242
          1       0.88      0.54      0.67       301

avg / total       0.93      0.94      0.93      2543

Feature 15: clothing
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      2521
          1       0.00      0.00      0.00        22

avg / total       0.98      0.99      0.99      2543

Feature 16: money
             precision    recall  f1-score   support

          

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [50]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as d:
    pickle.dump(classifier_model, d)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.